## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np 

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ribeira Grande,PT,38.5167,-28.7000,77.40,83,40,11.50,scattered clouds
1,1,Mar Del Plata,AR,-38.0023,-57.5575,55.87,86,100,4.76,overcast clouds
2,2,Hilo,US,19.7297,-155.0900,84.27,96,100,6.91,moderate rain
3,3,Avera,US,33.1940,-82.5271,92.71,39,39,2.21,scattered clouds
4,4,Tuktoyaktuk,CA,69.4541,-133.0374,40.91,95,100,14.97,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ribeira Grande,PT,38.5167,-28.7000,77.40,83,40,11.50,scattered clouds
2,2,Hilo,US,19.7297,-155.0900,84.27,96,100,6.91,moderate rain
9,9,Yellowknife,CA,62.4560,-114.3525,68.56,45,75,19.57,broken clouds
10,10,Guadix,ES,37.2993,-3.1392,82.45,26,3,4.79,clear sky
12,12,Thompson,CA,55.7435,-97.8558,64.56,55,75,8.05,broken clouds
13,13,Lompoc,US,34.6391,-120.4579,84.02,76,20,15.99,few clouds
18,18,Veliki Preslav,BG,43.1667,26.8167,69.51,59,0,4.63,clear sky
20,20,Carnarvon,AU,-24.8667,113.6333,66.69,54,76,20.02,broken clouds
22,22,San Pedro,PH,14.3500,121.0167,78.33,88,100,2.37,overcast clouds
26,26,Atuona,PF,-9.8000,-139.0333,77.25,78,45,18.90,light rain


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()


City_ID                417
City                   417
Country                416
Lat                    417
Lng                    417
Max Temp               417
Humidity               417
Cloudiness             417
Wind Speed             417
Current Description    417
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()
clean_df.count()


City_ID                416
City                   416
Country                416
Lat                    416
Lng                    416
Max Temp               416
Humidity               416
Cloudiness             416
Wind Speed             416
Current Description    416
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ribeira Grande,PT,77.40,scattered clouds,38.5167,-28.7000,
2,Hilo,US,84.27,moderate rain,19.7297,-155.0900,
9,Yellowknife,CA,68.56,broken clouds,62.4560,-114.3525,
10,Guadix,ES,82.45,clear sky,37.2993,-3.1392,
12,Thompson,CA,64.56,broken clouds,55.7435,-97.8558,
13,Lompoc,US,84.02,few clouds,34.6391,-120.4579,
18,Veliki Preslav,BG,69.51,clear sky,43.1667,26.8167,
20,Carnarvon,AU,66.69,broken clouds,-24.8667,113.6333,
22,San Pedro,PH,78.33,overcast clouds,14.3500,121.0167,
26,Atuona,PF,77.25,light rain,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

hotel_df.head(10)
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ribeira Grande,PT,77.40,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
2,Hilo,US,84.27,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
9,Yellowknife,CA,68.56,broken clouds,62.4560,-114.3525,The Explorer Hotel
10,Guadix,ES,82.45,clear sky,37.2993,-3.1392,Hotel YIT Abentofail
12,Thompson,CA,64.56,broken clouds,55.7435,-97.8558,Burntwood Hotel
13,Lompoc,US,84.02,few clouds,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
18,Veliki Preslav,BG,69.51,clear sky,43.1667,26.8167,семеен хотел Преслав
20,Carnarvon,AU,66.69,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
22,San Pedro,PH,78.33,overcast clouds,14.3500,121.0167,Palines Apartment and Guesthouse
26,Atuona,PF,77.25,light rain,-9.8000,-139.0333,Villa Enata


In [11]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df.loc[(hotel_df["Hotel Name"] ==""),"Hotel Name"] = np.nan

clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df.head(40)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ribeira Grande,PT,77.40,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
2,Hilo,US,84.27,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
9,Yellowknife,CA,68.56,broken clouds,62.4560,-114.3525,The Explorer Hotel
10,Guadix,ES,82.45,clear sky,37.2993,-3.1392,Hotel YIT Abentofail
12,Thompson,CA,64.56,broken clouds,55.7435,-97.8558,Burntwood Hotel
13,Lompoc,US,84.02,few clouds,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
18,Veliki Preslav,BG,69.51,clear sky,43.1667,26.8167,семеен хотел Преслав
20,Carnarvon,AU,66.69,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
22,San Pedro,PH,78.33,overcast clouds,14.3500,121.0167,Palines Apartment and Guesthouse
26,Atuona,PF,77.25,light rain,-9.8000,-139.0333,Villa Enata


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))